# Create New Spotify Playlists with Recommended Songs

In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time

In [4]:
# Set your Spotify credentials
SPOTIPY_CLIENT_ID = " "
SPOTIPY_CLIENT_SECRET = " "
SPOTIPY_REDIRECT_URI = "http://127.0.0.1:5000/callback"
  # Use this for testing

# Authenticate with Spotify
scope = "playlist-modify-public playlist-modify-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope=scope,
    open_browser=False
))


# Get your Spotify user ID
user_id = sp.current_user()["id"]

# Load the recommended songs dataset
df = pd.read_csv("./data/recommended_songs.csv")

# Ensure the column with track IDs exists
if "id" not in df.columns:
    raise ValueError("The dataset must contain a column with Spotify track IDs.")

Go to the following URL: https://accounts.spotify.com/authorize?client_id=83144fb5efda493aad91d5e789089417&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fcallback&scope=playlist-modify-public+playlist-modify-private
Enter the URL you were redirected to: http://127.0.0.1:5000/callback?code=AQBbqvucGA_RDLy3C8QaM7-uyJ4ZfhIjBxexHeDAUK-CrOIaOYlFaabBF2Mm-LLxNOhElvyDerd1SXm-zHKQ6FINrTDITdq7wgR3I31Y--0jJNngWJ7ixYPXG9gLDfID6FJv1_y6YYoYHgVaeLE3Wqs72Ka1jskTtziHJlG44xbBdFwAZ3Xix0LpQN6UJI_EROs26vJVnaiUSt9AJ_BDHJ12tPrOFE82ujfNQ6UUSIApMNU


In [5]:
def create_spotify_playlist(playlist_name, track_ids):
    """
    Creates a new playlist in Spotify and adds the given track IDs.
    """
    # Create playlist
    playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True)
    playlist_id = playlist["id"]
    
    # Add tracks in batches of 100 (Spotify limit)
    for i in range(0, len(track_ids), 100):
        sp.playlist_add_items(playlist_id, track_ids[i:i + 100])
        time.sleep(1)  # Avoid hitting API rate limits
    
    print(f"Playlist '{playlist_name}' created with {len(track_ids)} tracks.")

# Iterate over each unique playlist and add songs
for playlist in df["Playlist"].unique():
    playlist_tracks = df[df["Playlist"] == playlist]["id"].tolist()  # Spotify track IDs
    if playlist_tracks:
        create_spotify_playlist(playlist, playlist_tracks)


Playlist 'Cafe Contemplation' created with 5509 tracks.
Playlist 'Sunshine & Seratonin' created with 6418 tracks.
Playlist 'Upbeat & Moody' created with 2931 tracks.
